In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
from google.colab import drive 
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
train=pd.read_csv('/content/gdrive/My Drive/ML data sets /analytics india/Big Mart Sales Preds /train_v9rqX0R.csv')
test=pd.read_csv('/content/gdrive/My Drive/ML data sets /analytics india/Big Mart Sales Preds /test_AbJTz2l.csv')

In [ ]:
df = pd.concat([train, test])

In [ ]:
df['Item_Fat_Content'].replace({'low fat':'Low Fat','reg':'Regular','LF':'Low Fat'},inplace=True)

In [ ]:
#replaces nan values with mode of Outlet_Size
df['Outlet_Size'].fillna('Medium',inplace=True)

In [ ]:
df['Item_Weight'].fillna(12.792,inplace=True)

In [ ]:
df['Item_Identifier_num'] = df['Item_Identifier'].str[-2:]

In [ ]:
df['Item_Identifier_abc'] = df['Item_Identifier'].str[:-2]

In [ ]:
df['Item_Visibility_per'] = df['Item_Visibility']*100

In [ ]:
df.drop(['Outlet_Identifier','Item_Identifier','Item_Visibility'],axis=1,inplace=True)

In [ ]:
dummy_list=['Item_Fat_Content','Item_Type','Item_Identifier_abc','Outlet_Size','Outlet_Type','Outlet_Location_Type','Item_Identifier_abc','Item_Identifier_num']

In [ ]:
dummy_drop = []
clean_df = df
for i in dummy_list:
    dummy_drop += [i + '_' + str(df[i].unique()[-1])]

df = pd.get_dummies(df, columns = dummy_list)
df = df.drop(dummy_drop, axis = 1)

In [ ]:
df['item_qty_sold']=df['Item_Outlet_Sales']/df['Item_MRP']

In [ ]:
df.shape

(14204, 228)

In [ ]:
train_len = len(train)
train_ = df[:train_len]
test_ = df[train_len:]

In [ ]:
train_.shape

(8523, 228)

In [ ]:
test_.shape

(5681, 228)

random forest - item_qty_sold 

In [ ]:
from sklearn.model_selection import train_test_split,cross_val_score

In [ ]:
yi=train_['item_qty_sold'].values.reshape(-1,1)
xi=train_.drop(['item_qty_sold','Item_Outlet_Sales'],axis=1)
Xi_train,Xi_test,yi_train,yi_test = train_test_split(xi,yi,test_size=0.2,random_state=42)

In [ ]:
from sklearn.ensemble import RandomForestRegressor

In [ ]:
Random_forest_reg_01=RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=300, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=15,
                      min_samples_split=1000, min_weight_fraction_leaf=0.0,
                      n_estimators=190, n_jobs=None, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)

In [ ]:
Random_forest_reg_01.fit(Xi_train,yi_train.ravel())

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=300, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=15,
                      min_samples_split=1000, min_weight_fraction_leaf=0.0,
                      n_estimators=190, n_jobs=None, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)

In [ ]:
yi_pred = Random_forest_reg_01.predict(Xi_test)

In [ ]:
from sklearn import metrics

print('Mean Absolute Error:', metrics.mean_absolute_error(yi_test, yi_pred))
print('Mean Squared Error:', metrics.mean_squared_error(yi_test, yi_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(yi_test, yi_pred)))

Mean Absolute Error: 5.215156740212541
Mean Squared Error: 46.632797281286855
Root Mean Squared Error: 6.828821075506874


In [ ]:
from sklearn.svm import SVR
from sklearn.metrics import accuracy_score

clf = SVR(kernel='linear')


In [ ]:
clf.fit(Xi_train,yi_train.ravel())
y_pred = clf.predict(Xi_test)


In [ ]:
from sklearn import metrics

print('Mean Absolute Error:', metrics.mean_absolute_error(yi_test, yi_pred))
print('Mean Squared Error:', metrics.mean_squared_error(yi_test, yi_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(yi_test, yi_pred)))

Mean Absolute Error: 5.215156740212541
Mean Squared Error: 46.632797281286855
Root Mean Squared Error: 6.828821075506874


In [ ]:
y_pred

array([13.22856574, 17.84098323, 14.80135944, ..., 15.59461366,
       11.21768705, 13.21932758])

In [ ]:
test_.head()

,Item_Weight,Item_MRP,Outlet_Establishment_Year,Item_Outlet_Sales,Item_Visibility_per,Item_Fat_Content_Low Fat,Item_Type_Baking Goods,Item_Type_Breads,Item_Type_Breakfast,Item_Type_Canned,Item_Type_Dairy,Item_Type_Frozen Foods,Item_Type_Fruits and Vegetables,Item_Type_Hard Drinks,Item_Type_Health and Hygiene,Item_Type_Household,Item_Type_Meat,Item_Type_Others,Item_Type_Snack Foods,Item_Type_Soft Drinks,Item_Type_Starchy Foods,Item_Identifier_abc_DRA,Item_Identifier_abc_DRB,Item_Identifier_abc_DRC,Item_Identifier_abc_DRD,Item_Identifier_abc_DRE,Item_Identifier_abc_DRF,Item_Identifier_abc_DRG,Item_Identifier_abc_DRH,Item_Identifier_abc_DRI,Item_Identifier_abc_DRJ,Item_Identifier_abc_DRK,Item_Identifier_abc_DRL,Item_Identifier_abc_DRM,Item_Identifier_abc_DRN,Item_Identifier_abc_DRP,Item_Identifier_abc_DRQ,Item_Identifier_abc_DRY,Item_Identifier_abc_DRZ,Item_Identifier_abc_FDA,...,Item_Identifier_num_22,Item_Identifier_num_23,Item_Identifier_num_24,Item_Identifier_num_25,Item_Identifier_num_26,Item_Identifier_num_27,Item_Identifier_num_28,Item_Identifier_num_29,Item_Identifier_num_30,Item_Identifier_num_31,Item_Identifier_num_32,Item_Identifier_num_33,Item_Identifier_num_34,Item_Identifier_num_35,Item_Identifier_num_36,Item_Identifier_num_37,Item_Identifier_num_38,Item_Identifier_num_39,Item_Identifier_num_40,Item_Identifier_num_41,Item_Identifier_num_42,Item_Identifier_num_43,Item_Identifier_num_44,Item_Identifier_num_45,Item_Identifier_num_46,Item_Identifier_num_47,Item_Identifier_num_48,Item_Identifier_num_49,Item_Identifier_num_50,Item_Identifier_num_51,Item_Identifier_num_52,Item_Identifier_num_53,Item_Identifier_num_54,Item_Identifier_num_55,Item_Identifier_num_56,Item_Identifier_num_57,Item_Identifier_num_58,Item_Identifier_num_59,Item_Identifier_num_60,item_qty_sold
0,20.750,107.8622,1999,NaN,0.756484,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,NaN
1,8.300,87.3198,2007,NaN,3.842768,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN
2,14.600,241.7538,1998,NaN,9.957491,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,NaN
3,7.315,155.0340,2007,NaN,1.538839,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,NaN
4,12.792,234.2300,1985,NaN,11.859931,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN


In [ ]:
yi_pred_qty= Random_forest_reg_01.predict(test_)

In [ ]:
yi_pred_qty

array([15.63137129, 15.69294472,  2.43117564, ..., 16.96983073,
       17.08026651, 16.44263396])

In [ ]:
output = pd.DataFrame({'item_qty_sold': yi_pred_qty})

In [ ]:
test_final = pd.concat([test, output], axis=1, sort=False)

In [ ]:
test_final.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,item_qty_sold
0,FDW58,20.750,Low Fat,0.007565,Snack Foods,107.8622,OUT049,1999,Medium,Tier 1,Supermarket Type1,15.631371
1,FDW14,8.300,reg,0.038428,Dairy,87.3198,OUT017,2007,NaN,Tier 2,Supermarket Type1,15.692945
2,NCN55,14.600,Low Fat,0.099575,Others,241.7538,OUT010,1998,NaN,Tier 3,Grocery Store,2.431176
3,FDQ58,7.315,Low Fat,0.015388,Snack Foods,155.0340,OUT017,2007,NaN,Tier 2,Supermarket Type1,16.745317
4,FDY38,NaN,Regular,0.118599,Dairy,234.2300,OUT027,1985,Medium,Tier 3,Supermarket Type3,26.627881


In [ ]:
test_f = pd.concat([test_, output], axis=1, sort=False)

In [ ]:
test_f.head()

,Item_Weight,Item_MRP,Outlet_Establishment_Year,Item_Visibility_per,Item_Fat_Content_Low Fat,Item_Type_Baking Goods,Item_Type_Breads,Item_Type_Breakfast,Item_Type_Canned,Item_Type_Dairy,Item_Type_Frozen Foods,Item_Type_Fruits and Vegetables,Item_Type_Hard Drinks,Item_Type_Health and Hygiene,Item_Type_Household,Item_Type_Meat,Item_Type_Others,Item_Type_Snack Foods,Item_Type_Soft Drinks,Item_Type_Starchy Foods,Item_Identifier_abc_DRA,Item_Identifier_abc_DRB,Item_Identifier_abc_DRC,Item_Identifier_abc_DRD,Item_Identifier_abc_DRE,Item_Identifier_abc_DRF,Item_Identifier_abc_DRG,Item_Identifier_abc_DRH,Item_Identifier_abc_DRI,Item_Identifier_abc_DRJ,Item_Identifier_abc_DRK,Item_Identifier_abc_DRL,Item_Identifier_abc_DRM,Item_Identifier_abc_DRN,Item_Identifier_abc_DRP,Item_Identifier_abc_DRQ,Item_Identifier_abc_DRY,Item_Identifier_abc_DRZ,Item_Identifier_abc_FDA,Item_Identifier_abc_FDB,...,Item_Identifier_num_22,Item_Identifier_num_23,Item_Identifier_num_24,Item_Identifier_num_25,Item_Identifier_num_26,Item_Identifier_num_27,Item_Identifier_num_28,Item_Identifier_num_29,Item_Identifier_num_30,Item_Identifier_num_31,Item_Identifier_num_32,Item_Identifier_num_33,Item_Identifier_num_34,Item_Identifier_num_35,Item_Identifier_num_36,Item_Identifier_num_37,Item_Identifier_num_38,Item_Identifier_num_39,Item_Identifier_num_40,Item_Identifier_num_41,Item_Identifier_num_42,Item_Identifier_num_43,Item_Identifier_num_44,Item_Identifier_num_45,Item_Identifier_num_46,Item_Identifier_num_47,Item_Identifier_num_48,Item_Identifier_num_49,Item_Identifier_num_50,Item_Identifier_num_51,Item_Identifier_num_52,Item_Identifier_num_53,Item_Identifier_num_54,Item_Identifier_num_55,Item_Identifier_num_56,Item_Identifier_num_57,Item_Identifier_num_58,Item_Identifier_num_59,Item_Identifier_num_60,item_qty_sold
0,20.750,107.8622,1999,0.756484,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,15.631371
1,8.300,87.3198,2007,3.842768,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,15.692945
2,14.600,241.7538,1998,9.957491,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,2.431176
3,7.315,155.0340,2007,1.538839,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,16.745317
4,12.792,234.2300,1985,11.859931,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,26.627881


random forest final

In [ ]:
y=train_['Item_Outlet_Sales'].values.reshape(-1,1)
x=train_.drop(['Item_Outlet_Sales'],axis=1)
X_train,X_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=42)

In [ ]:
rf_model = RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=300, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=5,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=418, n_jobs=None, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)

In [ ]:
x.head()

,Item_Weight,Item_MRP,Outlet_Establishment_Year,Item_Visibility_per,Item_Fat_Content_Low Fat,Item_Type_Baking Goods,Item_Type_Breads,Item_Type_Breakfast,Item_Type_Canned,Item_Type_Dairy,Item_Type_Frozen Foods,Item_Type_Fruits and Vegetables,Item_Type_Hard Drinks,Item_Type_Health and Hygiene,Item_Type_Household,Item_Type_Meat,Item_Type_Others,Item_Type_Snack Foods,Item_Type_Soft Drinks,Item_Type_Starchy Foods,Item_Identifier_abc_DRA,Item_Identifier_abc_DRB,Item_Identifier_abc_DRC,Item_Identifier_abc_DRD,Item_Identifier_abc_DRE,Item_Identifier_abc_DRF,Item_Identifier_abc_DRG,Item_Identifier_abc_DRH,Item_Identifier_abc_DRI,Item_Identifier_abc_DRJ,Item_Identifier_abc_DRK,Item_Identifier_abc_DRL,Item_Identifier_abc_DRM,Item_Identifier_abc_DRN,Item_Identifier_abc_DRP,Item_Identifier_abc_DRQ,Item_Identifier_abc_DRY,Item_Identifier_abc_DRZ,Item_Identifier_abc_FDA,Item_Identifier_abc_FDB,...,Item_Identifier_num_22,Item_Identifier_num_23,Item_Identifier_num_24,Item_Identifier_num_25,Item_Identifier_num_26,Item_Identifier_num_27,Item_Identifier_num_28,Item_Identifier_num_29,Item_Identifier_num_30,Item_Identifier_num_31,Item_Identifier_num_32,Item_Identifier_num_33,Item_Identifier_num_34,Item_Identifier_num_35,Item_Identifier_num_36,Item_Identifier_num_37,Item_Identifier_num_38,Item_Identifier_num_39,Item_Identifier_num_40,Item_Identifier_num_41,Item_Identifier_num_42,Item_Identifier_num_43,Item_Identifier_num_44,Item_Identifier_num_45,Item_Identifier_num_46,Item_Identifier_num_47,Item_Identifier_num_48,Item_Identifier_num_49,Item_Identifier_num_50,Item_Identifier_num_51,Item_Identifier_num_52,Item_Identifier_num_53,Item_Identifier_num_54,Item_Identifier_num_55,Item_Identifier_num_56,Item_Identifier_num_57,Item_Identifier_num_58,Item_Identifier_num_59,Item_Identifier_num_60,item_qty_sold
0,9.30,249.8092,1999,1.604730,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,14.951963
1,5.92,48.2692,2009,1.927822,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,9.186454
2,17.50,141.6180,1999,1.676007,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,14.809346
3,19.20,182.0950,1998,0.000000,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4.021967
4,8.93,53.8614,1987,0.000000,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,18.467868


In [ ]:
rf_model.fit(X_train,y_train.ravel())

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=300, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=5,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=418, n_jobs=None, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)

In [ ]:
y_pred = rf_model.predict(X_test)

In [ ]:
from sklearn import metrics

print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

Mean Absolute Error: 23.0429097499139
Mean Squared Error: 4232.169075061362
Root Mean Squared Error: 65.05512335751398


In [ ]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

clf = SVC(kernel='linear')
clf.fit(x_train,y_train)
y_pred = clf.predict(x_test)
print(accuracy_score(y_test,y_pred))

In [ ]:
final_preds = rf_model.predict(test_f)

In [ ]:
final_preds

array([1665.91673595, 1373.6474987 ,  484.64400863, ..., 2041.28800092,
       3702.03357239, 1308.5558615 ])

In [ ]:
output_final = pd.DataFrame(final_preds)

In [ ]:
output_final

,0
0,1665.916736
1,1373.647499
2,484.644009
3,2532.658840
4,6274.259117
...,...
5676,2294.086222
5677,2357.012627
5678,2041.288001
5679,3702.033572


In [ ]:
test.columns

Index(['Item_Identifier', 'Item_Weight', 'Item_Fat_Content', 'Item_Visibility',
       'Item_Type', 'Item_MRP', 'Outlet_Identifier',
       'Outlet_Establishment_Year', 'Outlet_Size', 'Outlet_Location_Type',
       'Outlet_Type'],
      dtype='object')

In [ ]:
test.drop(['Item_Weight', 'Item_Fat_Content', 'Item_Visibility','Item_Type', 'Item_MRP','Outlet_Establishment_Year', 'Outlet_Size', 'Outlet_Location_Type',
       'Outlet_Type'],axis=1)

,Item_Identifier,Outlet_Identifier
0,FDW58,OUT049
1,FDW14,OUT017
2,NCN55,OUT010
3,FDQ58,OUT017
4,FDY38,OUT027
...,...,...
5676,FDB58,OUT046
5677,FDD47,OUT018
5678,NCO17,OUT045
5679,FDJ26,OUT017


In [ ]:
test.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type
0,FDW58,20.750,Low Fat,0.007565,Snack Foods,107.8622,OUT049,1999,Medium,Tier 1,Supermarket Type1
1,FDW14,8.300,reg,0.038428,Dairy,87.3198,OUT017,2007,NaN,Tier 2,Supermarket Type1
2,NCN55,14.600,Low Fat,0.099575,Others,241.7538,OUT010,1998,NaN,Tier 3,Grocery Store
3,FDQ58,7.315,Low Fat,0.015388,Snack Foods,155.0340,OUT017,2007,NaN,Tier 2,Supermarket Type1
4,FDY38,NaN,Regular,0.118599,Dairy,234.2300,OUT027,1985,Medium,Tier 3,Supermarket Type3


In [ ]:
 result_final = pd.concat([test, output_final], axis=1, sort=False)

In [ ]:
result_final

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,0
0,FDW58,20.750,Low Fat,0.007565,Snack Foods,107.8622,OUT049,1999,Medium,Tier 1,Supermarket Type1,1665.916736
1,FDW14,8.300,reg,0.038428,Dairy,87.3198,OUT017,2007,NaN,Tier 2,Supermarket Type1,1373.647499
2,NCN55,14.600,Low Fat,0.099575,Others,241.7538,OUT010,1998,NaN,Tier 3,Grocery Store,484.644009
3,FDQ58,7.315,Low Fat,0.015388,Snack Foods,155.0340,OUT017,2007,NaN,Tier 2,Supermarket Type1,2532.658840
4,FDY38,NaN,Regular,0.118599,Dairy,234.2300,OUT027,1985,Medium,Tier 3,Supermarket Type3,6274.259117
...,...,...,...,...,...,...,...,...,...,...,...,...
5676,FDB58,10.500,Regular,0.013496,Snack Foods,141.3154,OUT046,1997,Small,Tier 1,Supermarket Type1,2294.086222
5677,FDD47,7.600,Regular,0.142991,Starchy Foods,169.1448,OUT018,2009,Medium,Tier 3,Supermarket Type2,2357.012627
5678,NCO17,10.000,Low Fat,0.073529,Health and Hygiene,118.7440,OUT045,2002,NaN,Tier 2,Supermarket Type1,2041.288001
5679,FDJ26,15.300,Regular,0.000000,Canned,214.6218,OUT017,2007,NaN,Tier 2,Supermarket Type1,3702.033572


In [ ]:
result_final.to_csv('submission.csv', index=False)